In [2]:
os.environ['DISPLAY'] = 'localhost:10.0'

In [29]:
%matplotlib qt5
import mirnylib
import mirnylib.plotting
from mirnylib.numutils import observedOverExpected
from mirnylib.genome import Genome
import cooler
import dekker
import h5py

In [62]:
imp.reload(dekker)
dek = dekker.DekkerLib()

In [82]:
subset = dek.filter_lib({'cell type':'PFE','crosslinker':'FA'})
subset[['enzyme','%trans']]

,enzyme,%trans
ind,,
0,DdeI,45.35
1,DpnII,52.18
2,HindIII,38.39
3,NcoI,35.76


In [83]:
loops = dekker.get_loops(subset)

In [84]:
loops

,name,mask,totals
0,U54-PFE-FA-20161031-DdeI__08NOV16_PE50_C-Monster,"[[[451.0, 455.0, 457.0, 454.0, 452.0, 458.0, 4...","[[[383.845564103, 452.753935582, 385.545134354..."
1,U54-PFE-FA-DpnII-20161118__22NOV16_PE50_C-Monster,"[[[452.0, 453.0, 453.0, 451.0, 450.0, 456.0, 4...","[[[475.548243609, 397.711737747, 365.362027884..."
2,U54-PFE-FA-HindIII-20161206__13DEC16_C-Monster...,"[[[460.0, 455.0, 458.0, 457.0, 456.0, 459.0, 4...","[[[411.101617095, 361.01766396, 397.673730584,..."
3,U54-PFE-FA-NcoI-20161014__18OCT16_PE50_HCKFGBB,"[[[303.0, 304.0, 302.0, 306.0, 294.0, 286.0, 3...","[[[323.800944781, 279.192073471, 316.685854591..."


In [38]:
def factors(n, inv):
#    if all(n % i for i in range(2, n)):
    if n ==3:
        if inv:
            return (3,1)
        return (1,3)
    else:
        n_list = list(range(n,n+2))
        most_sq = []
        for num in n_list:
            a = list((i, num//i) for i in range(1, int(num**0.5) + 1) if num % i == 0)
            b = [y-x for x,y in a]
            most_sq.append(a[b.index(min(b))])

        b = [y-x for x,y in most_sq]
        most_sq = most_sq[b.index(min(b))]
        if inv:
            return most_sq[::-1]
        return most_sq
#     else:
#         a = list((i, n//i) for i in range(1, int(n**0.5) + 1) if n % i == 0)[-1]
#         if inv:
#             return a[::-1]
#        return a



def weight_func(matrix, power):
    lgth = len(matrix)
    if isinstance(power,(int, float)):
        power = float(power)
        ar = np.arange(0,lgth)
        wt_mat = 1 / (1 + np.abs(ar[:,None] - ar[None,:])) ** power
        matrix = matrix/wt_mat
        return matrix
            
    elif callable(power):
        return power(matrix)
    
    else:
        med = []
        for i, row in enumerate(matrix):
            if i-1 < 0:
                arr = row[(i+1):]
            elif i+1 > len(row)-1:
                arr = row[0:(i-1)]
            else:
                arr = np.concatenate((row[0:(i-1)],row[(i+1):]))
            med.append(arr.sum())
        matrix = matrix/np.median(med)
        return matrix
    
    
def lim_func(matrix, lim, func='max'):
    if isinstance(lim, (int,float)):
        return lim
    elif isinstance(lim, str):
        return np.diagonal(matrix, float(lim[1:])).mean()
    else:
        return None


def visualize(array, weight='', lim=(None,None), colormap='fall', log=True, inv=False):
    if log:
        f = np.log
    else:
        f = lambda x: x
        
    num = len(array)
    row,col = (4,12)#factors(num,inv)
    fig = plt.figure(figsize=(25,20))
    ax = [None]*num
    
    vmax = lim[0]
    vmin = lim[1]
    
#     title = []
#     ax_title_keys = []
#     for key in ['cell type','condition','crosslinker','enzyme']:
#         key_list = file_df[key].unique()
#         if len(key_list)==1:
#             if key_list[0] != '':
#                 title.append(key+': '+key_list[0])
#         else:
#             ax_title_keys.append(key)
#     title = ', '.join(title)
#     ax_title_keys+=['res', '%trans']
        
    for i, matrix in enumerate(array):
        
        matrix = weight_func(matrix, weight)
        vmax = lim_func(matrix, vmax,func='max')
        vmin = lim_func(matrix, vmin,func='min')
        if i==0:
            ax[i] = fig.add_subplot(row,col,i+1)#, facecolor='k')
        else:
            ax[i] = fig.add_subplot(row,col,i+1, sharex=ax[0], sharey=ax[0])#, facecolor='k')
        ax[i].set_adjustable('box-forced')
        ax[i].set_aspect(1)
#         ax_title = []
#         for key in ax_title_keys:
#             ax_title.append(key+': '+str(file[key]))
#         ax_title = ', '.join(ax_title)
#         ax[i].set_title(ax_title)

        ax[i].set_yticklabels([])
        ax[i].set_xticklabels([])
        im = ax[i].imshow(f(matrix), cmap=colormap, vmax=vmax, vmin=vmin)
        
    
    left = (row-1)*col
    right = row*col-1
    #ax[left].set_ylabel(r[0])
    
    cax = fig.add_axes([0.98, 0.25, 0.005, 0.5])
    fig.colorbar(im, cax=cax, orientation='vertical',ticklocation='left')
    

#     if weight == '':
#         fig.suptitle(title+'. Weighting: Median of marginals. Colorscale: '+('log'*log+'linear'*(not log)), fontsize=18, y=1)
#     elif weight == observedOverExpected:
#         fig.suptitle(title+'. Weighting: Observed over Expected. Colorscale: '+('log'*log+'linear'*(not log)), fontsize=18, y=1)
#     elif isinstance(weight,int):
#         fig.suptitle(title+'. Weighting: Diagonals weighted by s^'+int(weight)+' scaling. Colorscale: '+('log'*log+'linear'*(not log)), fontsize=18, y=1)
    fig.tight_layout(rect=(0,0,0.97,0.97), w_pad=1)
    plt.show()

In [86]:
for pu in loops['totals'].values:
    visualize(pu,log=False)

/home/sameer/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in true_divide
/home/sameer/miniconda3/lib/python3.6/site-packages/matplotlib/figure.py:1742: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
/home/sameer/miniconda3/lib/python3.6/site-packages/matplotlib/colors.py:821: UserWarning: Warning: converting a masked element to nan.
  dtype = np.min_scalar_type(value)
/home/sameer/miniconda3/lib/python3.6/site-packages/numpy/ma/core.py:2809: UserWarning: Warning: converting a masked element to nan.
  order=order, subok=True, ndmin=ndmin)


In [76]:
a = np.array([5,1])[-2:]
a

array([5, 1])

In [77]:
a.sort()

In [78]:
a

array([1, 5])

In [80]:
!ls /home/sameer/scalings_project/dekkerHires/coolers/loops/*PFE*FA*

/home/sameer/scalings_project/dekkerHires/coolers/loops/U54-PFE-FA-20161031-DdeI.5000.hdf5
/home/sameer/scalings_project/dekkerHires/coolers/loops/U54-PFE-FA-DpnII-20161118.5000.hdf5
/home/sameer/scalings_project/dekkerHires/coolers/loops/U54-PFE-FA-HindIII-20161206.5000.hdf5
/home/sameer/scalings_project/dekkerHires/coolers/loops/U54-PFE-FA-NcoI-20161014.5000.hdf5
